Universidad del Valle de Guatemala

Minería de Datos

Proyecto 2 Entrega 5

Wilson Calderón, 22018

Abby Donis, 22440


## 1. Creación de Variables Dicotómicas 

Se ha creado una variable categórica partiendo de SalePrice utilizando cuantiles del 33 y 66% respectivamente. Esto permite clasificar viviendas en tres grupos. Económico, Medio y Caro. De ahí se generaron variables binarias llamadas EsCara, EsMedia y EsEconomica que toman valores 1 o 0 según corresponda. Si una casa se encuentra en el tercil superior, la variable EsCara es 1. De lo contrario es 0.


In [ ]:
def main():
    # ---------------------
    # Inciso 1: Crear Variables Dicotómicas a partir de SalePrice
    # ---------------------
    try:
        df = pd.read_csv("train.csv")
    except Exception as e:
        print("Error al cargar 'train.csv':", e)
        return

    print("Columnas del dataset:", df.columns.tolist())

    # Utilizamos los cuantiles 33% y 66% para definir los umbrales
    quantiles = df['SalePrice'].quantile([0.33, 0.66])
    q_low = quantiles.iloc[0]
    q_high = quantiles.iloc[1]

    def categorizar_precio(x):
        if x >= q_high:
            return 'cara'
        elif x >= q_low:
            return 'media'
        else:
            return 'económica'

    df['Categoria'] = df['SalePrice'].apply(categorizar_precio)
    df['EsCara'] = (df['Categoria'] == 'cara').astype(int)
    df['EsMedia'] = (df['Categoria'] == 'media').astype(int)
    df['EsEconomica'] = (df['Categoria'] == 'económica').astype(int)

    print("\nPrimeras filas con SalePrice, Categoria y variables dicotómicas:")
    print(df[['SalePrice', 'Categoria', 'EsCara', 'EsMedia', 'EsEconomica']].head())

# 2. Mismos conjuntos de entrenamiento y prueba
Mismos conjuntos de entrenamiento y prueba
Para garantizar la reproducibilidad del experimento, el dataset train.csv se dividió en conjuntos de entrenamiento del 70% y prueba del 30%. Usando la función train_test_split con random_state = 42. Así se mantiene siempre las mismas particiones en cada ejecución.

In [ ]:
    # ---------------------
    # Inciso 3: Modelo de Regresión Logística con Validación Cruzada
    # ---------------------
modelo = LogisticRegression(max_iter=1000, solver='liblinear')
cv_scores = cross_val_score(modelo, X_train, y_train, cv=5, scoring='accuracy')
print("\nExactitud promedio en validación cruzada:", np.mean(cv_scores))
    
    # Entrena el modelo con el conjunto de entrenamiento completo
modelo.fit(X_train, y_train)

# 3. Modelo de regresión logística
Se utiliza un modelo de regresión logística para predecir si una vivienda es cara (variable EsCara). Este modelo ha sido entrenado con el conjunto entrenamiento y se aplicó validación cruzada (5-fold). Lo que produjo una exactitud promedio del 88.85%. Esto indica que el modelo se comporta de forma consistente.

In [ ]:
# ---------------------
# Inciso 3: Modelo de Regresión Logística con Validación Cruzada
# ---------------------
modelo = LogisticRegression(max_iter=1000, solver='liblinear')
cv_scores = cross_val_score(modelo, X_train, y_train, cv=5, scoring='accuracy')
print("\nExactitud promedio en validación cruzada:", np.mean(cv_scores))
    
# Entrena el modelo con el conjunto de entrenamiento completo
modelo.fit(X_train, y_train)

# 4. Análisis del modelo
Para evaluar el modelo se realizó un análisis de multicolinealidad mediante la matriz de correlación (ver imagen de abajo). Esta gráfica no muestra correlaciones excesivamente fuertes, por lo que es mejor para el modelo. Se examinaron OverallQual y GarageCars. Tienen coeficientes positivos y significativos, mientras que YearBuilt presenta un coeficiente negativo. La dirección y magnitud de los coeficientes, sin valores p formales, indican que algunas variables aportan de forma notable al modelo.



In [ ]:
# ---------------------
# Inciso 4: Análisis del Modelo 
# ---------------------
# 4.1: Análisis de multicolinealidad mediante la matriz de correlación
corr_matrix = X_train.corr()
plt.figure(figsize=(8, 6))
plt.imshow(corr_matrix, cmap='coolwarm', interpolation='none')
plt.colorbar()
plt.xticks(range(len(corr_matrix.columns)), corr_matrix.columns, rotation=90)
plt.yticks(range(len(corr_matrix.columns)), corr_matrix.columns)
plt.title("Matriz de correlación de variables predictoras")
plt.tight_layout()
plt.show()

# 5. Evaluación en el Conjunto de Prueba
Al aplicar el modelo sobre el conjunto de prueba se obtuvo una matriz de confusión con 282 verdaderos negativos, 122 verdaderos positivos, 14 falsos positivos y 20 falsos negativos. El reporte de clasificación muestra una exactitud global de 92.24%. Esto confirma que el modelo clasifica de forma eficiente. 


In [ ]:
# ---------------------
# Inciso 5: Evaluación del Modelo en el Conjunto de Prueba
# ---------------------
y_pred = modelo.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print("\nMatriz de Confusión:")
print(cm)
    
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred))
    
print("Exactitud (accuracy) en el conjunto de prueba:", accuracy_score(y_test, y_pred))

# 6. Sobreajuste y curvas de aprendizaje
Las exactitudes de la validación cruzada y  conjunto de prueba alcanzan un aproximado del 89% y 92.2% respectivamente. En conjunto con que el desempeño para la clase 0 la precisión y el recall llegan al 93% y 95%, en la clase 1 estos valores son de 90% y 86% respectivamente. Además de que el f1-score es del 94% y 88% para su clase respectiva, teniendo un buen balance. 
